In [9]:
from sklearn.model_selection import GridSearchCV, KFold
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_squared_error

# Definir el modelo
model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Definir el espacio de hiperparámetros para la búsqueda
param_grid = {
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 6],
    'subsample': [1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Configurar la búsqueda de hiperparámetros con validación cruzada
kf = KFold(n_splits=5, random_state=42, shuffle=True)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=kf, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

# Ejecutar la búsqueda de hiperparámetros
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_
print("Mejores hiperparámetros:", best_params)

# Entrenar el modelo con los mejores hiperparámetros
best_model = xgb.XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

# Predicción en el conjunto de prueba
y_pred_test = best_model.predict(X_test)

# Calcular el RMSE en el conjunto de prueba
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f"RMSE en el conjunto de prueba con los mejores hiperparámetros: {test_rmse:.4f}")


Fitting 5 folds for each of 24 candidates, totalling 120 fits
Mejores hiperparámetros: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'subsample': 1.0}
RMSE en el conjunto de prueba con los mejores hiperparámetros: 0.2917


In [7]:
# Preparar el conjunto de datos de submission
submission_data_path = "../data/processed/submission_data_processed_imputeKnn_scale.csv"  # Ruta de tu archivo de datos para predicción
submission_data = pd.read_csv(submission_data_path)
original_submission_data = pd.read_csv("../data/raw/submission_data.csv")

# Preprocesar el conjunto de datos de submission
# (Realiza las mismas transformaciones que hiciste para el conjunto de entrenamiento)
categorical_columns = submission_data.select_dtypes(include=['object']).columns
for col in categorical_columns:
    if col in label_encoders:  # Usa los label encoders existentes
        submission_data[col] = label_encoders[col].fit_transform(submission_data[col].astype(str))
submission_data = submission_data.fillna(0)

# Seleccionar las características utilizadas para el entrenamiento
X_submission = submission_data[features].values

# Predicciones con el modelo final
original_submission_data['prediction'] = best_model.predict(X_submission)

# Formatear el archivo de salida
original_submission_data = original_submission_data[['date', 'cluster_nl', 'prediction']]
original_submission_data.to_csv("../data/processed/submission_data_predictions_imputeKnn_scale_xgboost_hpt.csv", index=False)

print("Archivo de predicciones creado: submission.csv")

Archivo de predicciones creado: submission.csv


In [8]:
import pickle

# Save the model to a file in the models/weights folder
model_path = "../models/weights/model_imputeKnn_imputeKnn_scale_xgboost_hpt.pkl"
with open(model_path, 'wb') as file:
    pickle.dump(best_model, file)

print(f"Model saved to {model_path}")

Model saved to ../models/weights/model_imputeKnn_imputeKnn_scale_xgboost_hpt.pkl
